In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# MT5 Trade Analysis\n",
    "\n",
    "This notebook provides tools for analyzing past trades from MetaTrader 5 using Python. It includes functionality for:\n",
    "- Connecting to MT5\n",
    "- Retrieving historical trade data\n",
    "- Analyzing trade performance\n",
    "- Visualizing results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Import required libraries\n",
    "import MetaTrader5 as mt5\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from datetime import datetime, timedelta\n",
    "import pytz\n",
    "\n",
    "# Set plotting style\n",
    "plt.style.use('seaborn')\n",
    "sns.set_palette('husl')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. MT5 Connection Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def initialize_mt5():\n",
    "    \"\"\"Initialize connection to MT5 terminal\"\"\"\n",
    "    if not mt5.initialize():\n",
    "        print(\"initialize() failed\")\n",
    "        mt5.shutdown()\n",
    "        return False\n",
    "    return True\n",
    "\n",
    "def get_account_info():\n",
    "    \"\"\"Get account information\"\"\"\n",
    "    account_info = mt5.account_info()\n",
    "    if account_info is None:\n",
    "        print(\"Failed to get account info\")\n",
    "        return None\n",
    "    return account_info\n",
    "\n",
    "# Initialize MT5\n",
    "if initialize_mt5():\n",
    "    account_info = get_account_info()\n",
    "    if account_info:\n",
    "        print(f\"Connected to account #{account_info.login} {account_info.server}\")\n",
    "        print(f\"Balance: {account_info.balance}\")\n",
    "        print(f\"Equity: {account_info.equity}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Historical Trade Data Retrieval"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def get_trade_history(start_date=None, end_date=None):\n",
    "    \"\"\"\n",
    "    Retrieve trade history for a given date range\n",
    "    \n",
    "    Parameters:\n",
    "    -----------\n",
    "    start_date : datetime, optional\n",
    "        Start date for trade history\n",
    "    end_date : datetime, optional\n",
    "        End date for trade history\n",
    "    \n",
    "    Returns:\n",
    "    --------\n",
    "    pandas.DataFrame\n",
    "        DataFrame containing trade history\n",
    "    \"\"\"\n",
    "    if start_date is None:\n",
    "        start_date = datetime.now() - timedelta(days=30)\n",
    "    if end_date is None:\n",
    "        end_date = datetime.now()\n",
    "        \n",
    "    # Convert to UTC timezone\n",
    "    timezone = pytz.timezone(\"Etc/UTC\")\n",
    "    start_date = timezone.localize(start_date)\n",
    "    end_date = timezone.localize(end_date)\n",
    "    \n",
    "    # Get history\n",
    "    history = mt5.history_deals_get(start_date, end_date)\n",
    "    \n",
    "    if history is None:\n",
    "        print(\"Failed to get trade history\")\n",
    "        return None\n",
    "        \n",
    "    # Convert to DataFrame\n",
    "    df = pd.DataFrame(list(history), columns=history[0]._asdict().keys())\n",
    "    df['time'] = pd.to_datetime(df['time'], unit='s')\n",
    "    \n",
    "    return df\n",
    "\n",
    "# Example usage\n",
    "start_date = datetime.now() - timedelta(days=30)\n",
    "end_date = datetime.now()\n",
    "trade_history = get_trade_history(start_date, end_date)\n",
    "\n",
    "if trade_history is not None:\n",
    "    print(f\"Retrieved {len(trade_history)} trades\")\n",
    "    display(trade_history.head())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Trade Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def analyze_trades(trade_history):\n",
    "    \"\"\"\n",
    "    Analyze trade performance metrics\n",
    "    \n",
    "    Parameters:\n",
    "    -----------\n",
    "    trade_history : pandas.DataFrame\n",
    "        DataFrame containing trade history\n",
    "    \n",
    "    Returns:\n",
    "    --------\n",
    "    dict\n",
    "        Dictionary containing performance metrics\n",
    "    \"\"\"\n",
    "    if trade_history is None or len(trade_history) == 0:\n",
    "        return None\n",
    "        \n",
    "    # Calculate basic metrics\n",
    "    total_trades = len(trade_history)\n",
    "    winning_trades = len(trade_history[trade_history['profit'] > 0])\n",
    "    losing_trades = len(trade_history[trade_history['profit'] < 0])\n",
    "    \n",
    "    win_rate = winning_trades / total_trades if total_trades > 0 else 0\n",
    "    total_profit = trade_history['profit'].sum()\n",
    "    avg_profit = trade_history['profit'].mean()\n",
    "    \n",
    "    # Calculate drawdown\n",
    "    cumulative_profit = trade_history['profit'].cumsum()\n",
    "    rolling_max = cumulative_profit.expanding().max()\n",
    "    drawdown = (cumulative_profit - rolling_max) / rolling_max * 100\n",
    "    max_drawdown = drawdown.min()\n",
    "    \n",
    "    return {\n",
    "        'total_trades': total_trades,\n",
    "        'winning_trades': winning_trades,\n",
    "        'losing_trades': losing_trades,\n",
    "        'win_rate': win_rate,\n",
    "        'total_profit': total_profit,\n",
    "        'avg_profit': avg_profit,\n",
    "        'max_drawdown': max_drawdown\n",
    "    }\n",
    "\n",
    "# Analyze trades\n",
    "if trade_history is not None:\n",
    "    metrics = analyze_trades(trade_history)\n",
    "    if metrics:\n",
    "        print(\"\\nTrade Performance Metrics:\")\n",
    "        for key, value in metrics.items():\n",
    "            print(f\"{key}: {value:.2f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def plot_trade_analysis(trade_history):\n",
    "    \"\"\"\n",
    "    Create visualizations for trade analysis\n",
    "    \n",
    "    Parameters:\n",
    "    -----------\n",
    "    trade_history : pandas.DataFrame\n",
    "        DataFrame containing trade history\n",
    "    \"\"\"\n",
    "    if trade_history is None or len(trade_history) == 0:\n",
    "        return\n",
    "        \n",
    "    # Create figure with subplots\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "    \n",
    "    # 1. Cumulative Profit\n",
    "    cumulative_profit = trade_history['profit'].cumsum()\n",
    "    axes[0, 0].plot(trade_history['time'], cumulative_profit)\n",
    "    axes[0, 0].set_title('Cumulative Profit')\n",
    "    axes[0, 0].set_xlabel('Time')\n",
    "    axes[0, 0].set_ylabel('Profit')\n",
    "    \n",
    "    # 2. Profit Distribution\n",
    "    sns.histplot(data=trade_history, x='profit', ax=axes[0, 1])\n",
    "    axes[0, 1].set_title('Profit Distribution')\n",
    "    axes[0, 1].set_xlabel('Profit')\n",
    "    \n",
    "    # 3. Daily Profit\n",
    "    daily_profit = trade_history.groupby(trade_history['time'].dt.date)['profit'].sum()\n",
    "    axes[1, 0].bar(daily_profit.index, daily_profit.values)\n",
    "    axes[1, 0].set_title('Daily Profit')\n",
    "    axes[1, 0].set_xlabel('Date')\n",
    "    axes[1, 0].set_ylabel('Profit')\n",
    "    plt.xticks(rotation=45)\n",
    "    \n",
    "    # 4. Win/Loss Ratio\n",
    "    win_loss = pd.Series([\n",
    "        len(trade_history[trade_history['profit'] > 0]),\n",
    "        len(trade_history[trade_history['profit'] < 0])\n",
    "    ], index=['Wins', 'Losses'])\n",
    "    axes[1, 1].pie(win_loss, labels=win_loss.index, autopct='%1.1f%%')\n",
    "    axes[1, 1].set_title('Win/Loss Ratio')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "# Create visualizations\n",
    "if trade_history is not None:\n",
    "    plot_trade_analysis(trade_history)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Cleanup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# Shutdown MT5 connection\n",
    "mt5.shutdown()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}